In [69]:
#load data
import pandas as pd
from scipy import sparse
from collections import defaultdict
from scipy import sparse
from sklearn import model_selection as cv
import numpy as np

header = ["reviewerId","productId","rating","time"]
df = pd.read_csv('ratings_Video_Games.csv', names=header)
#df = pd.read_csv('item_dedup.csv', names=header)
n_users = df.reviewerId.unique().shape[0]
n_items = df.productId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))

# exclude data before this unix time : 1356955199 = 12/31/2012
print(len(df))

previousDF = df[df['time'] <= 1356955199]  
# exclude data after this unix time: 
afterDF = df[df['time'] > 1356955199]  
      
print(len(previousDF))
print(len(afterDF))



train_data, test_data = cv.train_test_split(previousDF, test_size=0)

# Create dict for product id 
#df.productId.unique()
productDict = defaultdict(dict)
userDict = defaultdict(dict)
count = 0


for id in train_data.reviewerId.unique():
    userDict[id] = count
    count += 1

count = 0
for id in train_data.productId.unique():
    productDict[id] = count
    count += 1
    
n_users = len(userDict)
n_items = len(productDict)

mat = sparse.lil_matrix((n_users, n_items),dtype=np.float32)
        
#print(productDict['B00004XOLO'])
#print(userDict['A2EHIE51TERF2C'])


for line in train_data.itertuples():
    mat[userDict[line[1]],productDict[line[2]]] = line[3]

print(mat.getnnz)
print("done")



Number of users = 826767 | Number of items = 50210
1324753
739884
584869
<bound method lil_matrix.getnnz of <457370x38568 sparse matrix of type '<class 'numpy.float32'>'
	with 739884 stored elements in LInked List format>>
done


In [70]:
# read metadata
import pandas as pd
import json
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i+= 1
    return pd.DataFrame.from_dict(df, orient='index')

metaDF = getDF('meta_Video_Games.json.gz')
# metaDF = pd.read_json("meta_Video_Games.json", orient='split')
print(metaDF)

             asin                                        description   price  \
0      0078764343                                  Brand new sealed!   37.98   
1      043933702X  In Stock NOW.  Eligible for FREE Super Saving ...   23.50   
2      0439339987  Grandma Groupers kelp seeds are missing and wi...    8.95   
3      0439342260  This software is BRAND NEW. Packaging may diff...     NaN   
4      0439339960               a scholastic clubs fairs cd rom game     NaN   
5      0439374391  CD-ROM: Thomas & Friends: The Great Festival A...   20.00   
6      0439394422  Product that encourages families to learn, exp...   12.96   
7      043940133X  Your ship has crash-landed on the planet Tatoo...   30.00   
8      0439573947                      3 Great games in one box set!   10.79   
9      0439591295                                    cartoon network     NaN   
10     0439591368  Travel to a remarkable new world and treat you...   19.95   
11     0439591538  WOW! 3d NANCY DREW MY

In [3]:
import requests
from bs4 import BeautifulSoup

# used when Nan not in metadata, note slower to get names so use at last step
def getProductTitleScrape(id):
    URL = "https://www.amazon.com/dp/" + id
    #print(URL)
    page = requests.get(URL,headers={"User-Agent":"Defined"})
    soup = BeautifulSoup(page.content, "html.parser")
    #print(soup)
    title = soup.find(id="productTitle").get_text().strip()
    print(title)

getProductTitleScrape("0439715571")



Mortal Kombat 4


In [71]:
#print(metaDF)
query = metaDF.loc[metaDF['asin'] == "B000007VDN"]
title = query['title'].iloc[0]
print(title)

def getProductTitle(asin):
    query = metaDF.loc[metaDF['asin'] == asin]
    title = str(query['title'].iloc[0])
    if title == "nan":
        print("Title not availiable for product " + str(asin))
    else:
        print(title)


getProductTitle("B003JKK2JC")


Underworld
Title not availiable for product B003JKK2JC


In [72]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing as pp

def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    #print(col_normed_mat)
    return col_normed_mat.T * col_normed_mat

def user_cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=1)
    #print(col_normed_mat)
    return col_normed_mat * col_normed_mat.T


#cosine_similarities(mat)
test = cosine_similarities(mat)
#userMatrix = user_cosine_similarities(mat)
#user_similarity = pairwise_distances(mat, metric='cosine')
print("done")
print(test.getnnz)
#print(userMatrix.getnnz)


done
<bound method _cs_matrix.getnnz of <38568x38568 sparse matrix of type '<class 'numpy.float32'>'
	with 5776718 stored elements in Compressed Sparse Row format>>


In [73]:
# https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices
#print(test[50208,40340])

#print(userMatrix.getrow(826766))

print("============")

row = test.getrow(6565).toarray()[0].ravel()
top_ten_indicies = row.argsort()[-6:]
top_ten_values = row[row.argsort()[-6:]]

print(top_ten_indicies)

print(top_ten_values)

#for x in top_ten_indicies:
#    print(list(productDict.keys())[list(productDict.values()).index(x)])
    


#for x in range(0,n_items):
#    print(x)
#    row = test.getrow(x).toarray()[0].ravel()
#    top_ten_values = (row[row.argsort()[-10:]])

print("done")



#array = test.getrow(i)
#print(array)


[30716 35631  8372 29472 25318  6565]
[0.21566555 0.21566555 0.21566555 0.25030056 0.5391639  1.        ]
done


In [240]:
def getTopFiveSimilarItems(itemId,rating):
    #print(itemId)
    row = test.getrow(itemId).toarray()[0].ravel()
    top_five_indicies = (row.argsort()[-11:])[:-1]
    top_five_values = (row[row.argsort()[-11:]])[:-1]
    weightedValues = list(map(lambda x: (x*rating/5), top_five_values))
    #print(top_five_values[:-1])
    #print(weightedValues)
    #print(list(top_five_indicies[:-1]))
    #myFormattedList = [ '%.2f' % elem for elem in  list(top_five_indicies[:-1]) ]
    #print(myFormattedList)
    resultList = []
    
    for i in range(0,10):
        resultList.append((weightedValues[i], top_five_indicies[i]))
    return resultList
    #returnList = list(zip(weightedValues, myFormattedList)
    #returnList = zip(weightedValues, list(top_five_indicies)
#     for x in top_five_indicies[:-1]:
#         #ignore same item
#         if (x != itemId):
#             #print(list(productDict.keys())[list(productDict.values()).index(x)])
#             #returnList.append(list(productDict.keys())[list(productDict.values()).index(x)])
#             #getProductTitleScrape(list(productDict.keys())[list(productDict.values()).index(x)])
#             #id = list(productDict.keys())[list(productDict.values()).index(x)]
#             #print(getProductTitle(id))


import time
start_time = time.time()
print(getTopFiveSimilarItems(9331,5))
print("--- %s seconds ---" % (time.time() - start_time))

#getTopFiveSimilarItems(productDict["B000EBC0PI"])
    

[(0.40422603487968445, 12354), (0.40422603487968445, 21552), (0.4373740255832672, 22460), (0.49507376551628113, 22391), (0.7001400589942932, 34922), (0.7001400589942932, 32354), (0.7001400589942932, 34693), (0.7001400589942932, 30499), (0.7001400589942932, 34545), (0.7756315469741821, 33444)]
--- 0.003996849060058594 seconds ---


In [237]:
# print(df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"])
# print("space")
# line = df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"]

def getUserItemRecommendation(userId):
    tempList = []
    returnList = []
    #line = previousDF.loc[df['reviewerId'] == userId]
    
    #improve performance compared to above
    string = f'reviewerId == {userId!r}'
    line = previousDF.query(string)
    #print(line)
    for x in line.iterrows():
        #print(x[1][1])
        #print(x[1][2])
        #print(productDict[x[1][1]])
        resultList = getTopFiveSimilarItems(productDict[x[1][1]],x[1][2])
        # concat list
        tempList.extend(resultList)
        
    #print(tempList)
    sortedTuples = sorted(tempList, reverse=True)
    #print(sortedTuples)
    # get top 5 items with highest cosine similarity values
    for value, index in sortedTuples[:10]:
         returnList.append(list(productDict.keys())[list(productDict.values()).index(index)])
    return returnList
    #print("======================")


#print(productDict["B007QYZ0SA"])
import time
start_time = time.time()
print(getUserItemRecommendation("A3SVQ324DA8CNM"))

#print(len(result))
print("--- %s seconds ---" % (time.time() - start_time))



# for x in line.iterrows():
#     #print(x[1][1])
#     #print(productDict[x[1][1]])
#     getTopFiveSimilarItems(productDict[x[1][1]])
#     #print("======================")
    
# df.loc[df['reviewerId'] == "A10RFAF1CGFKHZ"]
# print(list(productDict.keys())[list(productDict.values()).index(9331)]) 
# print(list(productDict.keys())[list(productDict.values()).index(23340)]) 
# print(list(userDict.keys())[list(userDict.values()).index(76565)]) 

['B00008XL0J', 'B00005ASGN', 'B003OPYQCG', 'B00006YXLE', 'B000ASBKL0', 'B00007UHL5', 'B004Y3951E', 'B0000945KD', 'B0000503QB', 'B002GC5YMY']
--- 0.1459190845489502 seconds ---


In [231]:
# check if recommended item was bought past the bounded time period

def getUserItemsAfter(userId):
    returnList = []
    #line = afterDF.loc[df['reviewerId'] == userId]
    #improved performance
    string = f'reviewerId == {userId!r}'
    line = afterDF.query(string)
    for x in line.iterrows():
        returnList.append(x[1][1])
    return returnList

#print(getUserItemsAfter("AFWPLXT2OD6H1"))
    
# count = 0
# accuracyList = []
# # get list of users in prev
# previousUserList = list(previousDF.reviewerId.unique())
# # iterate through list of all users in prev and after 
# print(previousUserList)
# print(previousUserList[0:9])
# for user in previousUserList:
#     #print(user)
#     recommendations = set(getUserItemRecommendation(user))
#     purchased = set(getUserItemsAfter(user))
#     accuracy = len(recommendations.intersection(purchased))/len(recommendations)
#     #print(accuracy)
#     if (accuracy > 0):
#         count += 1
#         print(accuracy)
#         accuracyList.append(accuracy)
#     #print(recommendations.intersection(purchased))

# print(count)
# print("done")


# L1 = set([2,3,4])
# L2 = set([1,2])
# print(len(L1.intersection(L2))/(len(L1)))

In [232]:
# Thread test
from threading import Thread
count = 0
accuracyList = []
threads = []
previousUserList = previousDF.reviewerId.unique()
import multiprocessing as mp


print(len(previousUserList))


def calculateAccuracy(userlist):
    for user in userlist:
        recommendations = set(getUserItemRecommendation(user))
        #print(len(recommendations))
        purchased = set(getUserItemsAfter(user))
        accuracy = len(recommendations.intersection(purchased))/len(recommendations)
        if (accuracy > 0):
            print(accuracy)
            accuracyList.append(accuracy)

# def test(userlist):
#     for x in userlist:
#         output.put(x)



# print(previousDF.reviewerId.unique().shape[0])
# splittedList = np.array_split(previousUserList, 4)

# for userlist in splittedList:
#     thread = Thread(target=calculateAccuracy, args=[userlist.tolist()])
#     thread.start()
#     threads.append(thread)
    
# for thread in threads:
#     thread.join()



         



457370


In [242]:
# import multiprocessing as mp

# def calc_acc(user):
#     print(f'Working on {user}')
#     recommendations = set(getUserItemRecommendation(user))
#     #print(len(recommendations))
#     purchased = set(getUserItemsAfter(user))
#     accuracy = min(len(recommendations.intersection(purchased)), 1)
#     return accuracy

# with mp.Pool(4) as p:
#     p.map(calc_acc, previousUserList[:12])

# write to csv file the recommendations

import time
start_time = time.time()

_u = previousUserList
#count = 1
for u in _u:
    #print(count/len(_u), end='\r')
    #count +=1
    with open('top-10-per-user.csv', 'a') as f:
        recs = [u] + getUserItemRecommendation(u)
        f.write(','.join(recs) + '\n')
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 67654.07045674324 seconds ---


In [220]:
# create user after csv
# import time
# start_time = time.time()

# _u = previousUserList
# count = 0
# for u in _u:
#     print(count/len(_u), end='\r')
#     count += 1
#     with open('userAfter.csv', 'a') as f:
#         recs = [u] + getUserItemsAfter(u)
#         f.write(','.join(recs) + '\n')
    
# print("--- %s seconds ---" % (time.time() - start_time))

--- 23784.486189365387 seconds ---


In [283]:
# open csv file
import csv

with open('top-10-per-user.csv', 'r') as f:
  reader = csv.reader(f)
  userItemRec = list(reader)

print(len(userItemRec))
# print(userItemRec[0][0])
# print(userItemRec[0][1:])

with open('userAfter.csv', 'r') as f:
  reader = csv.reader(f)
  userAfter = list(reader)

print(len(userItemRec))
print(len(userAfter))


457370
457370
457370


In [287]:
import time
start_time = time.time()

count1 = 0
count2 = 0
# use both files and get accuracy
for i in range(0,len(userAfter)):
    #print(counter)
        if (len(userAfter[i][1:])):
            count1 += 1
            if (set(userItemRec[i][1:]).intersection(set(userAfter[i][1:]))):
                count2 += 1


        
#for userItem in userItemRec:
#     recommendations = set(userItem[1:])
#     purchased = set(getUserItemsAfter(userItem[0]))
#     hit = recommendations.intersection(purchased)
#     if len(hit) > 0:
#         count += 1
#     if not set(userItem[1:]).intersection(set(getUserItemsAfter(userItem[0]))):
#         count += 1

accuracy = count2 / count1
print(accuracy)

print("--- %s seconds ---" % (time.time() - start_time))

0.010506377067811592
--- 0.23286700248718262 seconds ---


In [277]:
#test

#line = previousDF.loc[df['reviewerId'] == "A304YAM2CCDXNN"]
import time
start_time = time.time()
line = previousDF.loc[df['reviewerId'] == "A304YAM2CCDXNN"]
# user = "A304YAM2CCDXNN"
# string = f'reviewerId == {user!r}'
# line = previousDF.query(string)
#print(line)
#print(previousDF.query(string))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
user = "A304YAM2CCDXNN"
string = f'reviewerId == {user!r}'
line = previousDF.query(string)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.5982468128204346 seconds ---
--- 0.06196475028991699 seconds ---


set()
